In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List
from datetime import datetime
import requests
import socket
import logging
import json
import os
from dotenv import load_dotenv

load_dotenv()
os.makedirs("reports", exist_ok=True)
os.makedirs("logs", exist_ok=True)

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("logs/security_recon.log"), logging.StreamHandler()])
logger = logging.getLogger(__name__)

assert os.getenv("GEMINI_API_KEY"), "GEMINI_API_KEY not found"
print("="*60 + "\n🔐 SECURITY RECONNAISSANCE AGENT\n" + "="*60)

🔐 SECURITY RECONNAISSANCE AGENT


In [19]:
TARGET = input("Enter target domain: ").replace("https://", "").replace("http://", "").strip("/")
print(f"🎯 Target: {TARGET}")

🎯 Target: rkmvmfamily.in


In [20]:
print("\n📡 DNS RECONNAISSANCE\n" + "-"*40)

dns_info = {"A": [], "MX": [], "NS": [], "TXT": []}
try: dns_info["A"].append(socket.gethostbyname(TARGET))
except: pass

for t in ["MX", "NS", "TXT"]:
    try:
        r = requests.get(f"https://dns.google/resolve?name={TARGET}&type={t}", timeout=10).json()
        dns_info[t] = [a.get("data", "") for a in r.get("Answer", [])]
    except: pass

print(f"A: {dns_info['A']}\nMX: {dns_info['MX']}\nNS: {dns_info['NS']}\nTXT: {len(dns_info['TXT'])} records")


📡 DNS RECONNAISSANCE
----------------------------------------
A: ['208.91.198.171']
MX: ['10 mail.rkmvmfamily.in.']
NS: ['ns.rkmvmfamily.in.', 'ns1.plesk-web16.webhostbox.net.', 'ns2.plesk-web16.webhostbox.net.']
TXT: 2 records


In [22]:
print("\n🔍 SECURITY HEADERS\n" + "-"*40)

sec_headers = {"Strict-Transport-Security": "❌", "Content-Security-Policy": "❌", "X-Frame-Options": "❌",
               "X-Content-Type-Options": "❌", "X-XSS-Protection": "❌", "Referrer-Policy": "❌"}
server = {}

try:
    # Fixed: Removed the accidental URL paste from the arguments
    r = requests.get(f"https://{TARGET}", timeout=10, allow_redirects=True)
    for h in sec_headers:
        if h.lower() in [x.lower() for x in r.headers]: sec_headers[h] = "✅"
    server = {"Server": r.headers.get("Server", "Hidden"), "Status": r.status_code}
except Exception as e: print(f"Error: {e}")

for h, s in sec_headers.items(): print(f"{h}: {s}")
print(f"\nServer: {server}")


🔍 SECURITY HEADERS
----------------------------------------
Strict-Transport-Security: ❌
Content-Security-Policy: ❌
X-Frame-Options: ❌
X-Content-Type-Options: ❌
X-XSS-Protection: ❌
Referrer-Policy: ❌

Server: {'Server': '', 'Status': 403}


In [23]:
print("\n🔧 TECHNOLOGY DETECTION\n" + "-"*40)

technologies = []
try:
    r = requests.get(f"https://{TARGET}", timeout=10)
    content = r.text.lower() + str(r.headers).lower()
    sigs = {"WordPress": "wp-content", "React": "react", "Vue": "vue", "Angular": "angular",
            "jQuery": "jquery", "Next.js": "_next", "Nginx": "nginx", "Apache": "apache", "Cloudflare": "cf-ray"}
    technologies = [t for t, s in sigs.items() if s in content]
except: pass

print(f"Detected: {technologies if technologies else 'None'}")


🔧 TECHNOLOGY DETECTION
----------------------------------------
Detected: None


In [24]:
print("\n🌐 SUBDOMAIN ENUMERATION\n" + "-"*40)

subdomains = set()
try:
    r = requests.get(f"https://crt.sh/?q=%.{TARGET}&output=json", timeout=20)
    if r.ok:
        for e in r.json():
            for s in e.get("name_value", "").split("\n"):
                s = s.strip().lower()
                if s.endswith(TARGET) and "*" not in s: subdomains.add(s)
except: pass

for sub in ["www", "mail", "api", "admin", "dev", "app"]:
    try: socket.gethostbyname(f"{sub}.{TARGET}"); subdomains.add(f"{sub}.{TARGET}")
    except: pass

subdomains = sorted(list(subdomains))[:20]
print(f"Found {len(subdomains)}:")
for s in subdomains[:10]: print(f"  • {s}")


🌐 SUBDOMAIN ENUMERATION
----------------------------------------
Found 10:
  • archive.rkmvmfamily.in
  • magazine.rkmvmfamily.in
  • mail.rkmvmfamily.in
  • rkmvmfamily.in
  • shree.rkmvmfamily.in
  • test.rkmvmfamily.in
  • webmail.rkmvmfamily.in
  • www.rkmvmfamily.in
  • www.shree.rkmvmfamily.in
  • www.test.rkmvmfamily.in


In [25]:
print("\n🔎 GOOGLE DORKS\n" + "-"*40)

dorks = [f"site:{TARGET}", f"site:{TARGET} filetype:pdf", f"site:{TARGET} inurl:admin",
         f"site:{TARGET} inurl:login", f"site:{TARGET} inurl:api", f'site:{TARGET} intitle:"index of"',
         f"site:{TARGET} filetype:sql", f"site:{TARGET} inurl:.git"]

for i, d in enumerate(dorks, 1): print(f"{i}. {d}")


🔎 GOOGLE DORKS
----------------------------------------
1. site:rkmvmfamily.in
2. site:rkmvmfamily.in filetype:pdf
3. site:rkmvmfamily.in inurl:admin
4. site:rkmvmfamily.in inurl:login
5. site:rkmvmfamily.in inurl:api
6. site:rkmvmfamily.in intitle:"index of"
7. site:rkmvmfamily.in filetype:sql
8. site:rkmvmfamily.in inurl:.git


In [26]:
print("\n🤖 AI SECURITY ANALYSIS\n" + "-"*40)

class SecurityReport(BaseModel):
    risk_level: str = Field(description="LOW/MEDIUM/HIGH/CRITICAL")
    summary: str = Field(description="2-3 sentence summary")
    findings: List[str] = Field(description="Key findings")
    recommendations: List[str] = Field(description="Recommendations")
    attack_surface: List[str] = Field(description="Attack vectors")

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a senior penetration tester."),
    ("user", "Analyze this recon:\nTarget: {target}\nDNS: {dns}\nHeaders: {headers}\nTech: {tech}\nSubdomains: {subs}")
])

chain = prompt | model.with_structured_output(SecurityReport)
report = chain.invoke({"target": TARGET, "dns": json.dumps(dns_info), "headers": json.dumps(sec_headers),
                       "tech": technologies, "subs": len(subdomains)})

print(f"\n🎯 Risk: {report.risk_level}")
print(f"📝 {report.summary}")
print("\n🔍 Findings:")
for f in report.findings: print(f"  • {f}")
print("\n⚔️ Attack Surface:")
for a in report.attack_surface: print(f"  • {a}")
print("\n✅ Recommendations:")
for r in report.recommendations: print(f"  • {r}")

2026-02-06 21:43:44,590 - INFO - AFC is enabled with max remote calls: 10.



🤖 AI SECURITY ANALYSIS
----------------------------------------


2026-02-06 21:43:52,950 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"



🎯 Risk: HIGH
📝 The target website, rkmvmfamily.in, exhibits critical security misconfigurations due to the complete absence of essential HTTP security headers. This significantly increases the risk of client-side attacks such as cross-site scripting, clickjacking, and data exfiltration. The reliance on a shared hosting environment (Plesk) also introduces potential for broader vulnerabilities.

🔍 Findings:
  • Complete absence of critical HTTP security headers including Strict-Transport-Security, Content-Security-Policy, X-Frame-Options, X-Content-Type-Options, X-XSS-Protection, and Referrer-Policy.
  • The domain utilizes a shared hosting platform (Plesk), which can sometimes lead to shared vulnerabilities or less granular control over server configurations.
  • The SPF record uses a '~all' (softfail) mechanism, which is less strict than '-all' (hardfail) and could potentially allow for some email spoofing scenarios.
  • Ten subdomains were identified, each representing a potential ad

In [27]:
filename = f"reports/{datetime.now().strftime('%Y%m%d_%H%M%S')}_{TARGET.replace('.', '_')}.md"
with open(filename, "w") as f:
    f.write(f"# Security Report: {TARGET}\n\n**Risk:** {report.risk_level}\n\n")
    f.write(f"## Summary\n{report.summary}\n\n")
    f.write("## Findings\n" + "\n".join([f"- {x}" for x in report.findings]) + "\n\n")
    f.write("## Attack Surface\n" + "\n".join([f"- {x}" for x in report.attack_surface]) + "\n\n")
    f.write("## Recommendations\n" + "\n".join([f"- {x}" for x in report.recommendations]))

print(f"\n💾 Saved: {filename}")
print("\n" + "="*60 + "\n🔐 RECON COMPLETE\n" + "="*60)


💾 Saved: reports/20260206_214405_rkmvmfamily_in.md

🔐 RECON COMPLETE
